In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
df = pd.read_csv('diabetes.csv')

df.info()
df[['Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age']] = df[['Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age']].replace(0,np.NaN)
df.fillna(df.mean(), inplace = True)
print(df.eq(0).sum())

X = df[['Glucose','BMI','Age']]
y = df.iloc[:,8]


knn = KNeighborsClassifier(n_neighbors=19)
knn.fit(X, y)

import pickle
# #---save the model to disk---
filename = 'diabetes.sav'
# #---write to the file using write and binary mode---
pickle.dump(knn, open(filename, 'wb'))
# #---load the model from disk---
loaded_model = pickle.load(open(filename, 'rb'))

Glucose = 65
BMI = 70
Age = 50

prediction = loaded_model.predict([[Glucose, BMI, Age]])
print(prediction)
if (prediction[0]==0):
 print("Non-diabetic")
else:
 print("Diabetic")

[0]
Non-diabetic


C:\Users\huy\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


In [ ]:
import pickle
from flask import Flask, request, json, jsonify
import numpy as np

app = Flask(__name__)
#---the filename of the saved model---
filename = 'diabetes.sav'
#---load the saved model---
loaded_model = pickle.load(open(filename, 'rb'))
@app.route('/diabetes/v1/predict', methods=['POST'])
def predict():
 #---get the features to predict---
 features = request.json
 #---create the features list for prediction---
 features_list = [features["Glucose"],features["BMI"],features["Age"]]
 #---get the prediction class---
 prediction = loaded_model.predict([features_list])
 #---get the prediction probabilities---
 confidence = loaded_model.predict_proba([features_list])
 #---formulate the response to return to client---
 response = {}
 response['prediction'] = int(prediction[0])
 response['confidence'] = str(round(np.amax(confidence[0]) * 100 ,2))
 return jsonify(response)
if __name__ == '__main__':
 app.run(host='0.0.0.0', port=5000)